In [1]:
import pandas as pd
from pandarallel import pandarallel
import numpy as np

pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
import re

def preprocess_text(text):
    text = text.lower()
    text = re.sub("_", " ", text)
    text = " " + text + " "
    text = re.sub("\d{5,}", " ", text) # убираю длиньше чем есть в метках
    text = re.sub("\d+[\.\,]?\d* *(см|гр|г|л|мл|кг|шт|мкм|м)", " ", text) # убираю единицы измерения
    text = re.sub("\(.*\)", " ", text) # убираем скобки и содержимое
    text = re.sub("\[.*\]", " ", text) # убираем скобки и содержимое
    text = re.sub("<.+>", " ", text) # убираем скобки и содержимое
    text = re.sub("{.+}", " ", text) # убираем скобки и содержимое
    text = re.sub("\d+[\.\,]?\d*\s?%", " ", text) # процентное содержание в товарах

    text = re.sub("\x01", "е", text)
    for el in ["\x07", "\t", "\n", "\x18", "\x1a", '"', ",", "/", ":", ";", '\\\\', '{', '\|', '~', "\x7f", "\xa0", "°", "·", "є", "∙", "╣"]:
        text = re.sub(el, " ", text)

    for el in ["#", "\$", '%', '\(', '\)', '\*', "<", "=", ">", '\?', '\[', '\]', '\^', "¤", "ї", "ў", "┬", "╗", "░"]:
        text = re.sub(el, "", text)

    text = re.sub(" +", " ", text)
    return text.strip()

In [3]:
df = pd.read_csv("train_supervised_dataset.csv")

df.good = df.good.replace("лента.", "лента")
df.good = df.good.replace("товара нет", np.nan)
df.good = df.good.replace("т,а,б,л,е,т,к,и", "таблетки")
df.good = df.good.replace('автокормушка", "автопоилка', "автокормушка,автопоилка")

# df.name = df.name.parallel_apply(preprocess_text)
print(df.shape)
df.head()

(25000, 4)


,id,name,good,brand
0,0,Petmax Бантик леопард с красн розой 2шт,бантик,petmax
1,1,87191 Бусы для елки шарики_87191,бусы,NaN
2,2,Футболка Piazza Italia WR011446881,футболка,piazza italia
3,3,7) YI572-03X-ONE ЗАКОЛКА ДЛЯ ВОЛОС ДЛЯ ДЕВОЧКИ,заколка,NaN
4,4,Одежда (вес) 1500,одежда,NaN


In [4]:
# df_good = df[~df.good.isna()]
df_good = df[["name", "good"]].fillna("")
display(df_good.head())
print(df_good.shape)

# df_brand = df[~df.brand.isna()]
df_brand = df[["name", "brand"]].fillna("")
display(df_brand.head())
print(df_brand.shape)

,name,good
0,Petmax Бантик леопард с красн розой 2шт,бантик
1,87191 Бусы для елки шарики_87191,бусы
2,Футболка Piazza Italia WR011446881,футболка
3,7) YI572-03X-ONE ЗАКОЛКА ДЛЯ ВОЛОС ДЛЯ ДЕВОЧКИ,заколка
4,Одежда (вес) 1500,одежда


(25000, 2)


,name,brand
0,Petmax Бантик леопард с красн розой 2шт,petmax
1,87191 Бусы для елки шарики_87191,
2,Футболка Piazza Italia WR011446881,piazza italia
3,7) YI572-03X-ONE ЗАКОЛКА ДЛЯ ВОЛОС ДЛЯ ДЕВОЧКИ,
4,Одежда (вес) 1500,


(25000, 2)


In [5]:
def get_good_pos(row):
    if row["good"] == "": return (0, 0)
    start = row["name"].lower().find(row["good"])
    if start == -1:
        end = -1
    else:
        end = start + len(row["good"])

    return (start, end)

def get_brand_pos(row):
    if row["brand"] == "": return (0, 0)
    start = row["name"].lower().find(row["brand"])
    if start == -1:
        end = -1
    else:
        end = start + len(row["brand"])

    return (start, end)

In [6]:
df_good["good_pos"] = df_good.apply(get_good_pos, axis=1)
df_brand["brand_pos"] = df_brand.apply(get_brand_pos, axis=1)
df_good
# 24093

,name,good,good_pos
0,Petmax Бантик леопард с красн розой 2шт,бантик,"(7, 13)"
1,87191 Бусы для елки шарики_87191,бусы,"(6, 10)"
2,Футболка Piazza Italia WR011446881,футболка,"(0, 8)"
3,7) YI572-03X-ONE ЗАКОЛКА ДЛЯ ВОЛОС ДЛЯ ДЕВОЧКИ,заколка,"(17, 24)"
4,Одежда (вес) 1500,одежда,"(0, 6)"
...,...,...,...
24995,"Вода ""Саирме"" с/г 500мл",вода,"(0, 4)"
24996,Моя Семя . 0.175л и ассортим,,"(0, 0)"
24997,Рулет бисквитн.Яшкино клубничный со слив,рулет,"(0, 5)"
24998,460075794371 Почвогрунт Цветочное счастье Фас...,почвогрунт,"(13, 23)"


In [7]:
df_good[df_good.good_pos == (-1,-1)]
# 1588

,name,good,good_pos
25,"eвя нная ""Koрмушkа для птиц"" №02 фанера 8001...",кормушка,"(-1, -1)"
40,aтиk джу товый KД1 100% джут 10 м сурoвы...,канатик,"(-1, -1)"
45,Крaскa aкpиловая Ладoга 46 mл неаполитансk...,краска,"(-1, -1)"
59,"Бисeр Япония ""TОHO"" кpуглый 3 5 г №0129 op...",бисер,"(-1, -1)"
125,СЕРЕТИД МУЛЬТИДИСК ПОР. Д/ИНГ. ДОЗИР. 50 МКГ+1...,порошок,"(-1, -1)"
...,...,...,...
24821,"Kрaска аkpилoвая ""Ладогa"" 46 мл 2204606 Aр...",краска,"(-1, -1)"
24833,"Наполнит Фреш Степ 6,35кг тройн контр 18151",наполнитель,"(-1, -1)"
24893,2234322 Кошелёк весна-лето MARC JACOBS M,кошелек,"(-1, -1)"
24928,16 2456582343130 Мультфонар/ФОТО/2276,мультфонарик,"(-1, -1)"


In [8]:
df_good[df_good.good_pos == (0,0)]

,name,good,good_pos
12,ЧокоПай Дарк какао 6шт.Орион,,"(0, 0)"
57,Био-Баланс Киви-Виноград 1% 270г (БЗМЖ),,"(0, 0)"
98,Гербер пюре 130г нежные овощи с кроликом Гербер,,"(0, 0)"
105,Бехеровка ( Чехия) 50 1 порц.,,"(0, 0)"
107,Альпен-голд 85гр. Dark Темный с орехом,,"(0, 0)"
...,...,...,...
24945,00193659050073 : W NIKE DBREAK,,"(0, 0)"
24946,Мор. 48 копеек пломбир с малиной с напол.малин...,,"(0, 0)"
24966,авт л 32А Legrand,,"(0, 0)"
24990,Маргарита 40 см тесто пышное,,"(0, 0)"


In [9]:
# import regex
# regex.search("(кормушка){s<4}", 'eвя нная "koрмушkа для птиц" №02 фанера 8001')

In [10]:
df_brand[df_brand.brand_pos == (-1,-1)]
# 2450

,name,brand,brand_pos
10,"O`Stin / MT1Z51 01 L Джемпер Мужск., жемчужно-...",o'stin,"(-1, -1)"
18,"Чай Нести 1,5л черный лимон",nestea,"(-1, -1)"
45,Крaскa aкpиловая Ладoга 46 mл неаполитансk...,ладога,"(-1, -1)"
51,Ряженка 4% 400г стак. Судогда,судогодский молочный завод,"(-1, -1)"
57,Био-Баланс Киви-Виноград 1% 270г (БЗМЖ),био баланс,"(-1, -1)"
...,...,...,...
24962,Вода Аква Минерале 5л Негазированная,aqua minerale,"(-1, -1)"
24986,ОЗЕРА ШОКОЛАД 90Г,ozera,"(-1, -1)"
24991,Сырок глазированный Варёная Сгущёнка 40г (Кейко),keiko,"(-1, -1)"
24995,"Вода ""Саирме"" с/г 500мл",sairme,"(-1, -1)"


In [11]:
df_brand[df_brand.brand_pos == (0,0)]

,name,brand,brand_pos
1,87191 Бусы для елки шарики_87191,,"(0, 0)"
3,7) YI572-03X-ONE ЗАКОЛКА ДЛЯ ВОЛОС ДЛЯ ДЕВОЧКИ,,"(0, 0)"
4,Одежда (вес) 1500,,"(0, 0)"
6,"29-003 ОБОИ/ART/п.п. белые на флизе 1,06*10/ос...",,"(0, 0)"
11,Колготки дет.(6837 - 11),,"(0, 0)"
...,...,...,...
24985,обувь 680(шт),,"(0, 0)"
24987,Мороженое Рожок брусника КизК Пломбир 70 гр.,,"(0, 0)"
24989,Трос ДИНАМИЧЕСКИЙ РЫВКОВЫЙ А2ДМ буксировочный ...,,"(0, 0)"
24990,Маргарита 40 см тесто пышное,,"(0, 0)"


In [12]:
df_good[df_good.good_pos != (-1,-1)].to_csv("train_good.csv", index=False)

In [13]:
df_brand[df_brand.brand_pos != (-1,-1)].to_csv("train_brand.csv", index=False)